# Boson sampling Tutorial

In [75]:
import numpy as np
import strawberryfields as sf
import random as rd
from strawberryfields.ops import *

In [76]:
eng, q = sf.Engine(4)
FockList_ini = [1,1,0,1]

with eng:
    # prepare the input fock states
    Fock(FockList_ini[0]) | q[0]
    Fock(FockList_ini[1]) | q[1]
    Fock(FockList_ini[2]) | q[2] # vacuum state (optional)
    Fock(FockList_ini[3]) | q[3]

    # rotation gates
    Rgate(0.5719)
    Rgate(-1.9782)
    Rgate(2.0603)
    Rgate(0.0644)

    # beamsplitter array ()
    BSgate(0.7804, 0.8578)  | (q[0], q[1])
    BSgate(0.06406, 0.5165) | (q[2], q[3])
    BSgate(0.473, 0.1176)   | (q[1], q[2])
    BSgate(0.563, 0.1517)   | (q[0], q[1])
    BSgate(0.1323, 0.9946)  | (q[2], q[3])
    BSgate(0.311, 0.3231)   | (q[1], q[2])
    BSgate(0.4348, 0.0798)  | (q[0], q[1])
    BSgate(0.4368, 0.6157)  | (q[2], q[3])
    # end circuit
    # not performing measurement

# run the engine
state = eng.run('fock', cutoff_dim=7)

# extract the joint Fock probabilities
probs = state.all_fock_probs()

# print the joint Fock state probabilities
print(probs[1,1,0,1])
print(probs[2,0,0,1])

0.17468916048563937
0.10644192724642339


In [77]:
from numpy.linalg import multi_dot
from scipy.linalg import block_diag

let’s generate the matrix representing the unitary transformation of the input mode annihilation and creation operators

$$
R(\phi)\hat{a}= \hat{a} e^{i\phi} \\
BS(\theta, \phi)(\hat{a}_{1},\hat{a}_{2}) = \left[\begin{array}{c}
      t & -r^{*} \\
      r & t\\
    \end{array}\right] \quad
$$


In [78]:
Uphase = np.diag([np.exp(0.5719*1j),np.exp(-1.9782*1j),np.exp(2.0603*1j),np.exp(0.0644*1j)])

In [79]:
BSargs = [
(0.7804, 0.8578),
(0.06406, 0.5165),
(0.473, 0.1176),
(0.563, 0.1517),
(0.1323, 0.9946),
(0.311, 0.3231),
(0.4348, 0.0798),
(0.4368, 0.6157)
]

t_r_amplitudes = [(np.cos(q), np.exp(p*1j)*np.sin(q)) for q,p in BSargs]
# Official tutorial lack "*1j"
BSunitaries = [np.array([[t, -np.conj(r)], [r, t]]) for t,r in t_r_amplitudes]
# list of 2x2 numpy array

In [80]:
UBS1 = block_diag(*BSunitaries[0:2])
print(UBS1)
print('diagonaolize 2 arrays to make 4 x 4 unitary')

[[ 0.71063217+0.j         -0.46020324+0.53217938j  0.        +0.j
   0.        +0.j        ]
 [ 0.46020324+0.53217938j  0.71063217+0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.99794886+0.j
  -0.05566547+0.03161374j]
 [ 0.        +0.j          0.        +0.j          0.05566547+0.03161374j
   0.99794886+0.j        ]]
diagonaolize 2 arrays to make 4 x 4 unitary


In [81]:
UBS2 = block_diag([[1]], BSunitaries[2], [[1]])
UBS3 = block_diag(*BSunitaries[3:5])
UBS4 = block_diag([[1]], BSunitaries[5], [[1]])
UBS5 = block_diag(*BSunitaries[6:8])

In [82]:
U = multi_dot([UBS5, UBS4, UBS3, UBS2, UBS1, Uphase])
print('combine all unitaries\n')
print(U)

combine all unitaries

[[ 0.21954694-0.25653455j  0.61107685+0.52417894j -0.10270019+0.47447883j
  -0.02725023+0.03729095j]
 [ 0.45128186+0.60258291j  0.45695259+0.01230749j  0.13162587-0.45041774j
   0.03528319-0.05324427j]
 [ 0.03871009+0.49271556j -0.01921274-0.32184285j -0.24077647+0.52443283j
  -0.45838814+0.32963337j]
 [-0.15661908+0.22456857j  0.10999222-0.16375022j -0.42117984+0.18364484j
   0.81876918+0.06801566j]]


Calculate Permanate function in the analytic result of Boson sampling.

$$
|<n_{1},n_{2},…,n_{N}∣\psi^{'}>|^{2} =
\frac{|\mathrm{Perm}(U_{st})|^{2}}{m_{1}!m_{2}!,…,m_{N}!n_{1}!n_{2}!,…,n_{N}!}
$$

Note

It's classically hard to compute!

This function makes use of the Balasubramanian-Bax-Franklin-Glynn formula, which scales as $$O(2^{n−1}n^{2})$$
 for an n×n
 matrix.

In [5]:
def perm(M):
     n = M.shape[0]
     d = np.ones(n)
     j =  0
     s = 1
     f = np.arange(n)
     v = M.sum(axis=0)
     p = np.prod(v)
     while (j < n-1):
         v -= 2*d[j]*M[j]
         d[j] = -d[j]
         s = -s
         prod = np.prod(v)
         p += s*prod
         f[0] = 0
         f[j] = f[j+1]
         f[j+1] = j+1
         j = f[0]
     return p/2**(n-1)

Calculate Ust

Input Fock state has a photon in 0th, 1st, and 3rd mode.
$$|1,1,0,1> \to (0,1,3)$$
Output Fock state has 2 photons in 0th mode and a photon in 3rd mode.
$$|2,0,0,1> \to (0,0,3)$$

Photon number should be preserved!

In [84]:
clasic_calc = np.abs(perm(U[:,[0,1,3]][[0,0,3]]))**2 / 2
# / 2 derive to (1! * 1! * 0! * 1!) * (2! * 0! * 0! * 1!)
Boson_calc = probs[2,0,0,1]
print(clasic_calc, Boson_calc)

0.10644192724642332 0.10644192724642339


Ust計算の補足

In [54]:
A = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
print(A)
print('\n0, 1, 3列目を抜き出し')
print(A[:,[0,1,3]])
print('\n0, 0, 3列目を抜き出し')
print(A[:,[0,0,3]])
print('\n0, 1, 3列目を抜き出した行列から0,0,3行目を抜き出し')
print(A[:,[0,1,3]][[0,0,3]])

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]

0, 1, 3列目を抜き出し
[[ 1  2  4]
 [ 5  6  8]
 [ 9 10 12]
 [13 14 16]]

0, 0, 3列目を抜き出し
[[ 1  1  4]
 [ 5  5  8]
 [ 9  9 12]
 [13 13 16]]

0, 1, 3列目を抜き出した行列から0,0,3行目を抜き出し
[[ 1  2  4]
 [ 1  2  4]
 [13 14 16]]


## Input, Output Fock state をランダムに生成できるようにした

In [74]:
import numpy as np
import strawberryfields as sf
import random as rd
from strawberryfields.ops import *
from numpy.linalg import multi_dot
from scipy.linalg import block_diag
import math

In [75]:
def transform(photonNumList):
    resList = []
    for i in range(len(photonNumList)):
        for j in range(photonNumList[i]):
            resList.append(int(i))
    return resList

def perm(M):
     n = M.shape[0]
     d = np.ones(n)
     j =  0
     s = 1
     f = np.arange(n)
     v = M.sum(axis=0)
     p = np.prod(v)
     while (j < n-1):
         v -= 2*d[j]*M[j]
         d[j] = -d[j]
         s = -s
         prod = np.prod(v)
         p += s*prod
         f[0] = 0
         f[j] = f[j+1]
         f[j+1] = j+1
         j = f[0]
     return p/2**(n-1)

def Fact_Fock(FockList_ini, FockList_after):
    FockList_ini_Fact = [np.math.factorial(i) for i in FockList_ini]
    FockList_after_Fact = [np.math.factorial(i) for i in FockList_after]

    res = 1
    for i in range(len(FockList_ini)):
        res *= FockList_ini_Fact[i]

    for i in range(len(FockList_after)):
        res *= FockList_after_Fact[i]
    return res

In [76]:
NodeNum = 4
FockList_ini = np.zeros(4, dtype=int)
SumOfPhoton = 3

for i in range(SumOfPhoton):
    tmp = rd.randint(0, NodeNum-1)
    FockList_ini[tmp] += int(1)

eng, q = sf.Engine(4)

with eng:
    # prepare the input fock states
    Fock(FockList_ini[0]) | q[0]
    Fock(FockList_ini[1]) | q[1]
    Fock(FockList_ini[2]) | q[2] # vacuum state (optional)
    Fock(FockList_ini[3]) | q[3]

    # rotation gates
    Rgate(0.5719)
    Rgate(-1.9782)
    Rgate(2.0603)
    Rgate(0.0644)

    # beamsplitter array ()
    BSgate(0.7804, 0.8578)  | (q[0], q[1])
    BSgate(0.06406, 0.5165) | (q[2], q[3])
    BSgate(0.473, 0.1176)   | (q[1], q[2])
    BSgate(0.563, 0.1517)   | (q[0], q[1])
    BSgate(0.1323, 0.9946)  | (q[2], q[3])
    BSgate(0.311, 0.3231)   | (q[1], q[2])
    BSgate(0.4348, 0.0798)  | (q[0], q[1])
    BSgate(0.4368, 0.6157)  | (q[2], q[3])
    # end circuit
    # not performing measurement

# run the engine
state = eng.run('fock', cutoff_dim=7)

# extract the joint Fock probabilities
probs = state.all_fock_probs()

In [83]:
Uphase = np.diag([np.exp(0.5719*1j),np.exp(-1.9782*1j),np.exp(2.0603*1j),np.exp(0.0644*1j)])

BSargs = [
(0.7804, 0.8578),
(0.06406, 0.5165),
(0.473, 0.1176),
(0.563, 0.1517),
(0.1323, 0.9946),
(0.311, 0.3231),
(0.4348, 0.0798),
(0.4368, 0.6157)
]

t_r_amplitudes = [(np.cos(q), np.exp(p*1j)*np.sin(q)) for q,p in BSargs]
# Official tutorial lack "*1j"
BSunitaries = [np.array([[t, -np.conj(r)], [r, t]]) for t,r in t_r_amplitudes]
# list of 2x2 numpy array

UBS1 = block_diag(*BSunitaries[0:2])
UBS2 = block_diag([[1]], BSunitaries[2], [[1]])
UBS3 = block_diag(*BSunitaries[3:5])
UBS4 = block_diag([[1]], BSunitaries[5], [[1]])
UBS5 = block_diag(*BSunitaries[6:8])
U = multi_dot([UBS5, UBS4, UBS3, UBS2, UBS1, Uphase])


FockList_after = np.zeros(4, dtype=int)

for i in range(SumOfPhoton):
    tmp = rd.randint(0, NodeNum-1)
    FockList_after[tmp] += int(1)
  
print(list(FockList_ini))
print(list(FockList_after))

div = Fact_Fock(FockList_ini, FockList_after)

ini_vec = transform(FockList_ini)
after_vec = transform(FockList_after)
clasic_calc = np.abs(perm(U[:,ini_vec][after_vec]))**2 / div
Boson_calc = probs[FockList_after[0],
                   FockList_after[1],
                   FockList_after[2],
                   FockList_after[3]]
print(clasic_calc, Boson_calc)

[0, 2, 0, 1]
[0, 0, 1, 2]
0.006819330090093822 0.006819330090093822


2